# Maize Selection Trade-offs

Jupyter notebook describes ecohydrological model developed by Natasha Krell, Noah Spahn and Kelly Caylor to demonstrate crop water use of maize under various rainfall climatologies in central Kenya. The script below runs the model, simulates rainfall, outputs the results, and generates figures.

Last updated: 12 September 2019

## 1. Background
From Ridge to Reef notebook, Chapter 3.

### Tradeoffs between days to maturity and exposure to climate variability

This tradeoff is harder to quantify. Instead, we will model the interaction between days to maturity and crop failure. Our strategy will be to examine a very field-scale water balance that looks only at relative water availability \[mm/mm\], $s$, daily rainfall, $R$ \[mm/day\], and daily water demand, which we will characterize as reference evapotranspiration, $ET_0$ \[mm/day\]. Because temperature is relatively constant, we will assume that $ET_0$ is constant. Therefore our water balance is simply:


$$ \frac{dS}{dt} = R(t) - ET_0 $$

Note: need this equation to match what's in the model: dsdt(t) = R(t) - E(s,t) - L(s,t) - Q(s,t)

This formulation ignores many important processes related to soil water storage and plant response to drought. As defined, it is essentially a cumulative dryness index; during periods of the season when $\sum_t R(t) > t \times ET_0$, there will accumulation of water in the soil. During periods when $\sum_t R(t) < t \times ET_0$, the soil will dry out. 

Because the amount of plant available water in the soil in bounded between zero and some maximum, we add an additional parameter, $S_{max}$, which is the maximum amount of water `[mm]` that can be stored in the root zone. Now our model looks like this:

$$
\begin{eqnarray}
    \frac{dS}{dt} &=& R(t) - ET_0 & if & (0 \leq S \leq S_{max}) \\
    & & & else; \\
    \frac{dS}{dt} &=& 0 
\end{eqnarray}       
$$

Assuming a typical value of porosity is 0.4 and soil depth is 500 mm (0.5 meters), the value of $S_{max}$ is 200, here is what our model looks like in python:

```python
n = 0.4  # Porosity, [m3/m3]
Zr = 500 # Rooting depth [mm]
S_max = n*Zr    # Max soil water storage [mm]
S[0] = 30  # Initial soil water storage [mm]
ET_0 = 6.5 # Daily reference evapotranspiration [mm]

S = np.zeros(len(R)) # Pre-allocate the array for S
dSdt = np.zeros(len(R))# Pre-allocate the array for dSdt

for t in range(len(R)):
    dSdt[t] = R[t] - ET_0
    S[t+1] = S[t] + dSdt[t]
    if S[t+1] < S_max:
        S[t+1] = S_max
    if S[t+1] < 0:
        S[t+1] = 0
```

## 2. Import ecohydro model

Model is saved as `runmodel.py` in folder. Evaporation and transpiration are seperated to calculate ET. 

In [1]:
# import packages and set working directory
import numpy as np
import matplotlib.pyplot as plt
import os
#os.chdir('../maize-Toff')

# import objects
from farm import Climate
from farm import Soil
from farm import Crop
from farm import CropModel

In [2]:
# initialize objects
climate = Climate() # uses default climate values
soil = Soil('sand')
crop = Crop(soil=soil) # previously kc_max=1.2, LAI_max=2.0, T_max=4.0
#soil.set_nZr(crop)  
model = CropModel(crop=crop,soil=soil,climate=climate)

model.run()
model.output()

TypeError: calc_E() got an unexpected keyword argument 'plant'

### Model tests

Something is wrong with `s`, `theta`, or `psi`, which are all methods of Soil. To figure out which of these might be causing the issue, we can test various relative soil moisture values using this figure from Liao et al. 2001b showing the relationship between relative soil moisture (x-axis: low levels of soil moisture = 0 and max soil moisture = 1) and evapotranspiration (y-axis: the sum of losses from both plant transpiration and soil evaporation).

![Create figure 5 from Liao et al. 2001b](output/Liao2001bFig5.jpg)

 When soil moisture is between field capacity (s_fc) and 1, the soil moisture is sufficient and maximum ET, (E_max), is independent of soil moisture, (s). Between field capacity (s_fc) and S* (s_star), both plant transpiration and soil evaporation result in soil moisture losses. S* is the point at which soil water loss is due to both transpiration and evaporation before soil moisture levels decrease enough for transpiration to be reduced. When soil moisture is below S*, plants start reducing transpiration: they close their stomata to prevent water loss, and thus there is a linear decrease to wilting point (s_w) as photosynthesis-related transpiration and water uptake from roots slows until reaching wilting point. At wilting point there are very low levels of soil moisture, and there is only evaporation. Between wilting point and the hygroscopic point (s_h), soil water is depleteed only by evaporation.

1. Is the hygroscopic soil moisture (s_h) less than soil moisture at wilting point (s_w)?

In [ ]:
print(soil.sh)
print(crop.sw)

if soil.sh <= crop.sw:
    print("True")

2. Is the conversion from mm to MPa is correct for sw?
3. Is the conversion for s_star correct?

    **TODO:** need to add this calculation in.

In [ ]:
print(crop.sw)
print(crop.sw_MPa)

print(crop.s_star)
print(crop.s_star_MPa)

4. Is field capacity less than 1?

In [ ]:
if soil.sfc <= 1:
    print("True")

pow((soil.sfc - soil.sh)/(1-soil.sh),1)*100

5. Do the hygroscopic values correspond with expected values from Liao et al. 2001b, Figure 6: ca. 0.07 for loamy sand, and 0.19 for loam?

In [5]:
# Loamy sand
# initialize objects
climate = Climate() # uses default climate values
soil = Soil('loamy sand')
crop = Crop(soil=soil) 
soil.set_nZr(crop)
model = CropModel(crop=crop,soil=soil,climate=climate)
print(soil.sh)

0.12


In [3]:
# Loam
climate = Climate() # uses default climate values
soil = Soil('loam')

crop = Crop(soil=soil) 
soil.set_nZr(crop)
model = CropModel(crop=crop,soil=soil,climate=climate)

print(soil.sh)

0.24


In [ ]:
pow(0.05/(0.0417*60*24),1/(2*5.39+3))

7. What does our version of Figure 5 look like?

    **Note**: This is not what it should be like.

In [ ]:
# Plot s from model output
o = model.output()
o['e'].plot()

plt.title('Relative soil moisture in model.output')
plt.ylabel('Relative soil moisture, $\mathit{mm}$')
plt.xlabel('Time of season, $\mathit{d}$')
plt.show()

### Just messing around...

In [ ]:
# Check that the value of E_w (evaporation at wilting point) is less than E_max (max evapotranspiration)

# where to get E_w?
#climate.calc_E(soil.sh)

In [ ]:
soil.s

In [ ]:
soil.s(0.3)

In [ ]:
soil.s

# this is a method that returns the relative soil moisture value given the volumetric water content.
# the input is theta which is an interval between 0 and porosity = 0.395

soil.n

In [ ]:
# let's try and get linspace 0 to soil.n

PRECISION = 2

theta_values = np.linspace(0.1, soil.n, num=100).round(3)
theta_values
#len(theta_values)

#TODO: how to fix this: if I start with zero it throws an error below?

In [ ]:
vals = np.array([soil.s(theta_) for theta_ in theta_values])

d = np.arange(101)
plt.plot(vals, '-')
plt.title('Hmmm')
plt.xlabel('Hmm')
plt.show()

**Space to play around**

In [ ]:
soil = Soil('sand')
soil.set_nZr(crop)

In [ ]:
soil.calc_L(0.395)

In [ ]:
self = soil
self.Psi_L_MPa
self.theta(-1.5)
#self.n
#self.sfc
#self.psi(0.44*0.37) * 1E6 / 1000 / 9.8
pow(0.05/60/24/self.Ks,1/(2*self.b+3))*self.n

In [ ]:
self = soil

In [ ]:
# run for three maize varieties (early, med, late harvesting) and multiple seasons.
kc_collection = {}

for season_length in range(90, 120, 180):
  climate = Climate(t_seas=season_length)
  soil = Soil('sand')
  crop = Crop(soil=soil)
  soil.set_nZr(crop)
  model  = CropModel(crop=crop,soil=soil,climate=climate)
  model.run()
  kc_key = str(season_length)+"kc"
  kc_collection[kc_key] = model.kc
    
# now want to repeat this process but for multiple rainfall climatologies, soil types, different output variables, etc. etc.

In [ ]:
kc_collection

In [ ]:
model.n_days

In [ ]:
climate = Climate()
soil

In [ ]:
climate.calc_E(0.3, 1, soil=soil, plant=crop)

from math import exp
climate.ET_max*exp(-0.5*crop.calc_LAI(1)/crop.LAI_max)

(0.3-soil.sh)/(1-soil.sh)
soil.theta(-1000)

crop.s_star

crop.s_star_MPa


In [ ]:
soil.Psi_S_MPa

In [ ]:
soil = Soil('loamy sand')
soil.theta(-10)/soil.n


**Playing around with calc_LAI**

In [ ]:
crop = Crop(soil=soil)

crop.calc_LAI(100)

self = crop
self.T_max

self.sw

In [ ]:
def calc_LAI(self, day_of_season, p=1):
        """ Returns a Leaf Area Index (LAI) variable. LAI comes
            from function of kc. Currently based on linear relationship 
            between kc and LAI (assumption).
        
        Usage: calc_LAI(t, p=1)

        LAI = (LAI_max/kc_max)^p * kc(t),

        where kc varies through the season according to calc_kc(t) 

        Note: p=1 assumes a linear relationship between LAI and kc

        """
        return pow((self.LAI_max/self.kc_max),p) * self.calc_kc(day_of_season)


In [ ]:
#self.calc_LAI(1)

p=1
day_of_season = 1
pow((self.LAI_max/self.kc_max),p) * self.calc_kc(day_of_season)


In [ ]:
self.calc_LAI(1)

In [ ]:
d = np.arange(121)
plt.plot(model.LAI, '-')
plt.title('Relationship between LAI and Time of Season')
plt.xlabel('Time of Season, $\mathit{t}$')
plt.show()

plt.plot(model.LAI, model.kc)
plt.title('Relationship between Kc and LAI')
plt.xlabel('LAI')
plt.ylabel('Kc')

## 3. Simulatate rainfall totals

In [ ]:
alpha=10
lambda_r = 0.3
t_seas = 120
n_seasons = 500
rainfall = np.array(calc_R(alpha, lambda_r, t_seas, n_seasons=500))

calc_R(10, 0.2, 120, 1)

In [ ]:
plt.hist(rainfall.sum(1))

### Non-stationarity

Adding non-stationary in "yearly alphas". Not sure where this should go. Perhaps in function with figures.
Previous = 500 realizations, not necessarily 500 years. Now: how to vary for multiple years.

In [ ]:
lambda_start = 0.3
lambda_end = 0.2
lambda_values = np.linspace(lambda_start, lambda_end, num=50)
rainfall = np.array([calc_R(alpha, lambda_r, t_seas) for lambda_r in lambda_values])
plt.plot(rainfall.sum(2))

### Code from Ridge to Reef Chapter 2

Using a $\lambda_r$ value of <code>0.40</code>, and assuming that the length of a growing season, $T_{seas}$, is 100 days, simulate a season of rainfall "events", where the value of a day is <code>1</code> if rainfall occurs, and <code>0</code> if not.

In [ ]:
T_seas = 100 # Days in each season
N_seas = 100 # Number of seasons to simulate
alpha = 11
s_mat = np.random.uniform(low=0, high=1, size=[N_seas, T_seas])
amounts = np.random.exponential(scale=alpha, size=[N_seas, T_seas])

lam = 0.4 # Lambda value
rain_days = np.array(s_mat <= lam).astype(int)
rain = rain_days * amounts
rain.sum(axis=1).std()/rain.sum(axis=1).mean() # Analytical equation for this!

# Remains to be done: Need to include _seasonal_ variation in lambda and maybe(?) alpha.

### Prepare historical rainfall using CETRAD dataset

In [ ]:
from datetime import datetime
from dateutil.relativedelta import *

df = pd.read_csv("../data/CETRAD/CETRAD_rainfall.csv")  # Read in the raw csv data.

# Step 1. Convert text strings into datetime objects.
format = '%m/%d/%y' # Column RDate has data in M/D/YY
df['Datetime']=pd.to_datetime(df['RDate'], format=format) # Create a new column of datetime objects using RDate.

# 2. Step 2. Convert future dates inferred during the conversion back into 20th century dates.
# Python is a future-looking programming language, and assumes that 1/1/34 is Jan 1, 2034.
# We can fix this by finding all the dates in the future (dt > datetime.now()) and removing 100 years from
# their value. This requires using the relativedelta function, which handles weird stuff like leap years.
df['Datetime'] = df['Datetime'].map(lambda dt: dt+relativedelta(years=-100) if dt > datetime.now() else dt)

# Step 3. Extract the Year and Month from the Datetime to make aggregation easier.
df['Year'] = [dt.year for dt in df['Datetime']]
df['Month'] = [dt.month for dt in df['Datetime']]

# Step 4. Use the Datetime values as the index for this dataframe.
df = df.set_index(pd.DatetimeIndex(df['Datetime']))  # Set the Datetime column as the dataframe index

# Step 5.  Delete the old RDate column, which we no longer need. 
# We will keep the Datetime column, in case we need it later.
df = df.drop(['RDate'], axis=1)

In [ ]:
df.head()
df.tail()
station_list = [station for station in df.columns if station not in ['Year', 'Day', 'Month', 'Datetime']]
print(station_list)

Determine mean annual rainfall for each station

In [ ]:
# Import cleaned, merged datasets

In [ ]:
AnnualRainfall = pd.read_csv("../data/CETRAD/AnnualRainfall.csv")  # Read in the Annual Rainfall per station
AnnualRainfall

In [ ]:
md = pd.read_csv("../data/CETRAD/CETRADMetadata.csv")  # Read in the each stations metadata
md.head()

In [ ]:
md.describe()

In [ ]:
# Return min, max, med:
md.sort_values(by='Annual Mean Rainfall')

# stations to use:
# L: OL JOGI FARM: 1967-1999, 32 years
# M: Kisima Farm
# H: Timau Marania: 1951-2015, # 64 years

### Step 1. Filter data

In [ ]:
station = 'ARDENCAPLE FARM' # 67 years of data
#station = 'JACOBSON FARM'
#station = 'EMBORI FARM'
columns = [station] + ['Year', 'Month', 'Datetime']
rainfall = df[columns]
rainfall.head()

Check out the data

In [ ]:
# Plot a histogram of rainfall values for days with rain.
daily_rainfall = rainfall.loc[rainfall[station] > 0][station]
daily_rainfall.plot.hist()

This looks okay, but we need to check and see how well the values in `daily_rainfall` fit our assumption of an exponential distribution.

### Step 2. Fit the distribution

To fit the distribution, we are going to use some more functions from `python`'s suite of numerical analysis. In this case we are going to use some functions from `scipy`. The [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html) module has a large suite of distribution functions pre-defined, which we can use to develop a fit for our data. Using any of these distributions for fitting our data is very easy. The distribution we are most interested in is the exponential distribution, which is called `expon` in the `stats` module.

In [ ]:
import scipy.stats as st

distribution = st.expon
data = daily_rainfall
params = distribution.fit(data, loc=0) # Force the distribution to be built off of zero

print(params)

arg = params[:-2]
loc = params[-2]
scale = params[-1]

### Step 3. Calculate fitted PDF and error with fit in distribution

To test the fit of our distribution, we can compare the empirical histogram to that predicted by our model. To do this, we first use our `data` to generate the empirical histogram. In this exampkle, we break the data into `30` bins, and we generate a histrogram of `density` rather than counts. This allows for an easier comparison between our empirical data and the fitted probability distribution function. Here are the steps:

1. Generate a histogram, from the `data`. Save the bin locations in `x` and the density of values in `y`
2. Shift the `x` bin locations generated from the histogram to the center of bins.
3. Calculate the value of the fitted `pdf(x)` for each of the bins in `x`.
4. Determine the residual sum of the squares, $SS_{error}$, and total sum of squares, $SS_{yy}$, according to:

$$ SS_{error} = \sum_{i=1}^{n} \left(y_i - f(x_i)\right)^2 $$
$$ SS_{yy} = \sum_{i=1}^{n} \left(y_i - \bar{y}\right)^2 $$

5. Calculate the $r^2$ of the fit, according to

$$ r^2 = 1- \frac{SS_{error}}{SS_{yy}} $$ 

 

In [ ]:
# Step 1. Generate a density histogram of the data 
y, x = np.histogram(data, bins=30, density=True)

# Step 2. Shift the x bin locations to the center of bins.
x = (x + np.roll(x, -1))[:-1] / 2.0

# Step 3. Calculate the values of pdx(x) for all x.
pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)

# Step 4. Determine the residual and total sum of the squares.
ss_error = np.sum(np.power(y - pdf, 2.0))
ss_yy = np.sum(np.power(y - y.mean(), 2.0))

r_2 = 1 - ( ss_error / ss_yy )
print(r_2)


This is an extremely good fit, so we can be confident that our assumption about an exponential distribution of rainfall is reasonable.

## Determining monthly values of  $\lambda_r$ 

We did something very similar to this in `Chapter 1`. Basically, we need to determine the probability of rainfall for each month by dividing the number of rainy days per month by the total number of observations in each month. For now, we assume stationarity in the monthly values, which means that we are assuming that the values of $\lambda_r$ in each month are the same through out the entire record (i.e. Jan 1938 has the same properties as Jan 2008). 

<div class="alert alert-info">💡 It's worth thinking about how you could test our stationarity assumption. If you have an idea of how to do so, go ahead and give it a shot!</div>

As a first step, let's get all the `rain_days` and all of the `observation_days` from the data on `JACOBSON FARM`. We will use method chaining to run a **groupby()** as we go.

In [ ]:
# First, find all the rows in the data where it rained and group by month.
rain_days = rainfall.loc[rainfall[station] > 0]

# Find all locations in the data where an observation was made.
all_days = rainfall.loc[rainfall[station] >= 0]

rain_days.head()

In [ ]:
lambda_by_month = (
    rain_days.groupby('Month')[station].count() /
    all_days.groupby('Month')[station].count()
    )
pd.DataFrame(lambda_by_month).plot.bar()

In [ ]:
lambda_by_year = (
    rain_days.groupby('Year')[station].count() /
    all_days.groupby('Year')[station].count()
    )
pd.DataFrame(lambda_by_year).plot.bar()

## Simulating annual rainfall

To simulate annual rainfall, we are going to specify daily values of $\lambda_r$ using the monthly values we just calculated. The use of a variable $\lambda$ value in a poisson process creates what is known as an "inhomogenous poisson process" (or, alternatively, "nonhomogeneous"... unforatunetly, there isn't much homogeneity in what we call it!). These types of processes allow the properties of the process to change in space and time. Our implementation - using monthly values - is a little clunky, and we'd prefer to have the $\lambda$ values change more smoothly throughout the year. However, we probably don't have sufficient data to allow for this, even if we could accomodate the more complicated coding it would require. 

In order to generate our nonhomogenous process we will first generate a daily array of month numbers for the year `2018`. This is really easy in python using `datetime` + `timedelta` (which we need to import).

```python
    import datetime
    from datetime import timedelta
    datetimes = np.arange(datetime(2018,1,1), datetime(2018,12,31), timedelta(days=1)).astype(datetime)
    month_value = np.array([datetime.month for datetime in datetimes])
```

In [ ]:
from datetime import timedelta, datetime
datetimes = np.arange(datetime(2018,1,1), datetime(2018,12,31), timedelta(days=1)).astype(datetime)
month_value_by_day = np.array([datetime.month for datetime in datetimes])

## TODO: turn this into dekad_value_by_day

In [ ]:
print(lambda_by_month)

## TODO #: turn this into lambda_by_dekad
## TODO: turn this into plot, and plot for each station 

In [ ]:
print(month_value_by_day)

`numpy` makes it really easy to map the values in one array onto the values of another.

```python
    lambda_values = [lambda_by_month[i] for i in month_value_by_day]
```

In [ ]:
lambda_values = np.array([lambda_by_month[i] for i in month_value_by_day])

With daily values of $\lambda_r$, we only need to follow the same cookbook we used to make the seasonal data:

```python
    alpha = scale  # Let's use the value we estimated from our exponential fit.
    simulated_rainfall_values = np.random.exponential(
        scale=alpha, size=len(lambda_values)) # Use the len() function instead of a constant.
```

    

In [ ]:
alpha = scale

simulated_rainy_days = (np.random.uniform(low=0, high=1, size=len(lambda_values)) <= lambda_values).astype(int)
simulated_rainfall_values = np.random.exponential(scale=alpha, size=len(lambda_values))

simulated_rainfall = simulated_rainy_days * simulated_rainfall_values

## Plotting our simulated data

We can easily plot our rainfall data using the `datetimes` object we already created as the basis of our x-axis.

In [ ]:
fig, ax = plt.subplots()

ax.bar(datetimes, simulated_rainfall, align='center', alpha=0.5)

<div class="alert alert-success">✏️ <strong>DIY Coding</strong>: Play around with our simulation. Here are a couple of ideas:</div>

1. Run 100 simulations of rainfall with our model for `JACOBSON FARM`. Compare the distribution of annual rainfall to the empirical distribution of annual rainfall.

2. Run the model for the same number of years that are in the `JACOBSON FARM` record and compare the mean and standard deviations of annual rainfall between the simulation and the empirical data. How well do they match?

2. Using the same simulations as in `2`, compare the mean and standard deviations of monthly rainfall between the simulation and the empirical data.

3. We've made $\lambda_r$ change every month, but we've used a constant $\alpha$ value. Determine if the assumption of a constant $\alpha$ is reasonable.

4. Determine the model parameters for a different station, preferably one with much lower or higher rainfall. How do the values of $\alpha$ and $\lambda_r$ change between stations?

5. Earlier I touched on the idea of non-stationarity in which the values of $\lambda_r$ and/or $\alpha$ may be changing over time. If you're up for it, see if you can come up with a way to investigate if our assumption of stationarity is valid.


## 4. Plotting

In [ ]:
# Relationship between s and E
N = 10
y = np.zeros(N)
rs = np.linspace(0, 1, N, endpoint=True); rs

# Cacluate E_max before plotting calc_E
# Currently E_max is set as 1 and 2: need to fix.
calc_E_max(1,1)

# plot calc_E
plt.plot(rs, [calc_E(s,1) for s in rs], '-')
plt.plot(rs, [calc_E(s,2) for s in rs], '--')
plt.title('Effect of LAI on evaporation as a function of relative soil moisture')
plt.xlabel('Relative soil moisture content, $\mathit{s}$')
plt.ylabel('Evaporation, $\mathit{E}$')
plt.legend(['LAI = 5','LAI = 0'])
#plt.show()
#plt.axhline(y=2, color='black', linewidth=1.5)

#calc_E(4,1)

In [ ]:
# Relationship between s and T
calc_T(0.1,1)

N = 100
y = np.zeros(N)
tr = np.linspace(0, 1, N, endpoint=True)

# Calculate T_max (which requires E_max) before plotting calc_T
plt.plot(tr, [calc_T(s,1) for s in tr], '-')
plt.title('Relationship between relative soil water availability and transpiration')
plt.xlabel('Relative soil moisture content, $\mathit{s}$')
plt.ylabel('Transpiration, $\mathit{T(s)}$')
#plt.show()

In [ ]:
# Relationship between kc and DOY
N = 120
y = np.zeros(N)
yx = np.linspace(0, 120, N, endpoint=False)

plt.plot(yx, [calc_kc(x) for x in yx], '-')
#plt.show()